### QUANTITATIVE VALUE AND MOMENTUM FACTOR STRATEGY

Value factor identifies stocks trading below their intrinsic value, using metrics like price-to-earnings (P/E), price-to-book (P/B), price-to-sales (P/S), enterprise value to EBITDA (EV/EBITDA), and enterprise value to gross profit (EV/GP). These metrics evaluate a stock's price relative to earnings, assets, sales, or profitability. Momentum factor focuses on stocks with the highest recent price momentum, assuming that stocks that have been performing well recently will continue to perform well in the short-term. This strategy targets stocks with the lowest valuation metrics, assuming they are undervalued and likely to outperform and exhibit strong, high-quality momentum across various timeframes (1-month, 3-months, 6-months, and 1-year)

In [ ]:
import numpy as np
import pandas as pd
import yfinance as yf
import math
from scipy import stats 
import warnings
from statistics import mean 
from datetime import datetime, timedelta
import time

warnings.filterwarnings("ignore")

stocks = ["ABG.JO", "ADH.JO", "AEL.JO", "AFE.JO", "AFH.JO", "AFT.JO", "AGL.JO", "AHR.JO", "AIP.JO", "ANG.JO", "ANH.JO", "APN.JO", "ARI.JO",
          "ARL.JO", "ATT.JO", "AVI.JO", "BAW.JO", "BHG.JO", "BID.JO", "BLU.JO", "BOX.JO", "BTI.JO", "BTN.JO", "BVT.JO", "BYI.JO", "CFR.JO", "CLS.JO",
          "CML.JO", "COH.JO", "CPI.JO", "CSB.JO", "DCP.JO", "DRD.JO", "DSY.JO", "DTC.JO", "EMI.JO", "EQU.JO", "EXX.JO", "FBR.JO", "FFB.JO", "FSR.JO",
          "FTB.JO", "GFI.JO", "GLN.JO", "GND.JO", "GRT.JO", "HAR.JO", "HCI.JO", "HDC.JO", "HMN.JO", "HYP.JO", "IMP.JO", "INL.JO", "INP.JO", "ITE.JO",
          "JSE.JO", "KAP.JO", "KIO.JO", "KRO.JO", "KST.JO", "LHC.JO", "LTE.JO", "MCG.JO", "MKR.JO", "MNP.JO", "MRP.JO", "MSP.JO", "MTH.JO", "MTM.JO",
          "MTN.JO", "N91.JO", "NED.JO", "NPH.JO", "NPN.JO", "NRP.JO", "NTC.JO", "NY1.JO", "OCE.JO", "OMN.JO", "OMU.JO", "OUT.JO", "PAN.JO", "PHP.JO",
          "PIK.JO", "PMR.JO", "PPC.JO", "PPH.JO", "PRX.JO", "QLT.JO", "RBX.JO", "RCL.JO", "RDF.JO", "REM.JO", "RES.JO", "RLO.JO", "RNI.JO", "S32.JO",
          "SAC.JO", "SAP.JO", "SBK.JO", "SHC.JO", "SHP.JO", "SLM.JO", "SNT.JO", "SOL.JO", "SPG.JO", "SPP.JO", "SRE.JO", "SRI.JO", "SSS.JO","SSU.JO", 
          "SSW.JO", "SUI.JO", "TBS.JO", "TFG.JO", "TGA.JO", "TKG.JO", "TRU.JO", "TSG.JO", "VAL.JO", "VKE.JO", "VOD.JO", "WBC.JO", "WHL.JO"]

**Explanation**

We use numpy for calculations, pandas for data handling, yfinance for Yahoo Finance data, math for share calculations, scipy.stats for percentiles, and xlswriter for Excel output. The warnings library suppress yfinance depreciation warnings.

In [ ]:
from IPython.display import display 

def get_valuation_ratios(ticker):
    stock = yf.Ticker(ticker)
    info = stock.info

    try:
        name = info.get('shortName', 'N/A')
        sector = info.get('sector', 'N/A')
        price_cents = info.get('currentPrice', np.nan)
        price = price_cents / 100 if not np.isnan(price_cents) else np.nan
        price_str = f"R{price:,.2f}" if not np.isnan(price) else "N/A"
        
        pe_ratio = info.get('trailingPE', np.nan)
        pb_ratio = info.get('priceToBook', np.nan)
        if np.isnan(pb_ratio):
            print(f"P/B is missing for {ticker}")
        ps_ratio = info.get('priceToSalesTrailing12Months', np.nan)
        ev = info.get('enterpriseValue', np.nan)
        ebitda = info.get('ebitda', np.nan)
        gross_profit = info.get('grossProfits', np.nan)
        ev_to_ebitda = ev / ebitda if ev and ebitda else np.nan
        ev_to_gp = ev / gross_profit if ev and gross_profit else np.nan

        return {
            'Ticker': ticker,
            'Name': name,
            'Sector': sector,
            'Price': price_str,
            'P/E': pe_ratio,
            'P/B': pb_ratio,
            'P/S': ps_ratio,
            'EV/EBITDA': ev_to_ebitda,
            'EV/GP': ev_to_gp
        }

    except Exception as e:
        print(f"Error fetching ratios for {ticker}: {e}")
        return {
            'Ticker': ticker,
            'Name': 'N/A',
            'Sector':'N/A',
            'Price': 'N/A',
            'P/E': np.nan,
            'P/B': np.nan,
            'P/S': np.nan,
            'EV/EBITDA': np.nan,
            'EV/GP': np.nan
        }

value_data = [get_valuation_ratios(ticker) for ticker in stocks]
value_df = pd.DataFrame(value_data)

value_metrics = ['P/E', 'P/B', 'P/S', 'EV/EBITDA', 'EV/GP']
for col in value_metrics:
    value_df[col] = pd.to_numeric(value_df[col], errors='coerce').round(0).astype('Int64')

end_date = datetime.today()
start_date = end_date - timedelta(days=730)

price_data = yf.download(stocks, start=start_date, end=end_date)['Close']
momentum_df = pd.DataFrame(index=stocks)

lookback_periods = {
    '1M': 21,
    '3M': 63,
    '6M': 126,
    '1Y': 252,
}

for label, days in lookback_periods.items():
    returns = price_data.pct_change(periods=days).iloc[-1]
    momentum_df[f"{label} Return"] = returns

for label in lookback_periods.keys():
    momentum_df[f"{label} Return"] = momentum_df[f"{label} Return"].apply(lambda x: f"{x:.2%}")

combined_df = pd.merge(value_df, momentum_df, left_on='Ticker', right_index=True)
pd.set_option('display.width', None)
pd.set_option('display.max_columns', None)
display(combined_df)

combined_df.to_excel("stock_valuation_momentum.xlsx", index=False)

**Explanation**

We use yfinance to fetch stock data (price, trailing P/E, P/B, P/S, enterprise value, EBIT, EBITDA, gross profit) and calculate EV/EBIT, EV/EBITDA, and EV/GP. A DataFrame is initialized with columns for tickers, prices, shares to buy, metrics, percentiles, and a robust value (RV) score. Error handling ensures robustness if data is missing. Combining data fetching and DataFrame creation streamlines the data collection process. Also, the historical price data will allow us to calculate momentum returns over different time periods. 

In [ ]:
valuation_columns=['P/E', 'P/B', 'P/S','EV/EBITDA', 'EV/GP']

momentum_columns = ['1M Return','3M Return','6M Return','1Y Return']

for column in valuation_columns:
    combined_df[column] = combined_df[column].astype(float)
    combined_df[column].fillna(combined_df[column].mean(), inplace=True)

for column in momentum_columns:
    combined_df[column] = combined_df[column].str.rstrip('%').astype(float) / 100
    combined_df[column].fillna(combined_df[column].mean(), inplace=True)

print("Missing values per column:")
print(combined_df.isnull().sum())

**Explanation**

We replace missing values with the mean of non-missing values for each metric. This preserves the dataset's size, assuming missing data is not systematically biased. This step remains standalone due to its distinct focus on data cleaning.

In [ ]:
value_metrics = {
    'P/E': 'PE Percentile',
    'P/B': 'PB Percentile',
    'P/S': 'PS Percentile',
    'EV/EBITDA': 'EV/EBITDA Percentile',
    'EV/GP': 'EV/GP Percentile'
}

momentum_metrics = {
    '1M Return': '1M Percentile',
    '3M Return': '3M Percentile',
    '6M Return': '6M Percentile',
    '1Y Return': '1Y Percentile'
}

for row in combined_df.index:
    for metric, pct_col in value_metrics.items():
        combined_df.loc[row, pct_col] = stats.percentileofscore(
            combined_df[metric], combined_df.loc[row, metric]
        ) / 100

for row in combined_df.index:
    for metric, pct_col in momentum_metrics.items():
        combined_df.loc[row, pct_col] = stats.percentileofscore(
            combined_df[metric], combined_df.loc[row, metric]
        ) / 100

combined_df['Value Score'] = combined_df[list(value_metrics.values())].mean(axis=1)
combined_df['Momentum Score'] = combined_df[list(momentum_metrics.values())].mean(axis=1)

value_buy_thresh = combined_df['Value Score'].quantile(0.2)
value_sell_thresh = combined_df['Value Score'].quantile(0.8)
momentum_buy_thresh = combined_df['Momentum Score'].quantile(0.8)
momentum_sell_thresh = combined_df['Momentum Score'].quantile(0.2)

combined_df['Value Signal'] = combined_df['Value Score'].apply(
    lambda x: 'BUY' if x <= value_buy_thresh else ('SELL' if x >= value_sell_thresh else 'HOLD')
)

combined_df['Momentum Signal'] = combined_df['Momentum Score'].apply(
    lambda x: 'BUY' if x >= momentum_buy_thresh else ('SELL' if x <= momentum_sell_thresh else 'HOLD')
)

final_df = combined_df[['Ticker', 'Value Score', 'Value Signal', 'Momentum Score', 'Momentum Signal']]
final_df.sort_values(by='Ticker', inplace=True)
final_df.reset_index(drop=True, inplace=True)

pd.set_option('display.max_rows', None)
print(final_df) 

final_df.to_excel("value_momentum_signals.xlsx", index=False)                                                                                        

**Explanation**

We compute percentile ranks for each metric using scipy.stats.percentile of score.  The RV Score is the average of the valuation metrics, with lower scores indicating better value, and the average of the momentum metrics, with higher scores indicating higher quality momentum. We sort by RV Score, select the top stocks, and reset the index. Combining these steps aligns the analytical focus on ranking and filtering. 

The quantitative value and momentum factor strategy is designed to identify and capitalize on two distinct market inefficiencies: undervaluation (via the value factor) and price trends (via the momentum factor). The screener aims to buy low (cheap) value stocks, sell high (expensive) value stocks, buy high positive momentum stocks, and sell low negative momentum stocks. .

### DISCOUNTED CASH FLOW (DCF) VALUATION MODEL

The DCF model estimates a company's intrinsic value by projecting its future cash flows and discounting them to the present value using the Weighted Average Cost of Capital (WACC). The model assumes that a company's value is the sum of its future Free Cash Flow to Firm (FCFF), adjusted for the time value of money and risk.

To estimate the intrinsic value of a company, several components and financial metrics are required. FCFF represents the cash flow available to all capital providers - both equity and debt  - after accounting for operating expenses, taxes, capital expenditures, and changes in working capital. The Weighted Average Cost of Capital (WACC) is used as the discount rate in valuation, incorporating the cost of equity and cost of debt, each weighted according to the company's capital structure, to reflect its risk profile. 

Another critical metric is Return on Invested Capital (ROIC), which measures how efficiently a company generates returns on the capital invested in its business determining its quality. Comparing ROIC to WACC helps assess whether the firm is creating or destroying value. The intrinsic value is determined by discounted projected FCFFs and a terminal value (which captures the value beyond the explicit forecast period using a perpetual growth rate), then subtracting net debt and dividing the results by the number of shares outstanding to obtain a fair value per share. 

The necessary data to perform these calculations is sourced from Yahoo Finance and includes financial line items such as operating income (EBIT), taxes payable, depreciation and amortization, capital expenditures, changes in non-cash working capital, interest expense, total debt, income before tax, market capitalization, number of shares outstanding, cash balances, and beta. In addition to these, certain assumptions must be made - such as the risk-free rate, expected market return, short-term FCCF growth rate, and perpetual growth rate - as they are not directly available from financial databases. 

With this data, several calculations are performed: determining the cost of debt and cost of equity (using the Capital Asset Pricing Model or CAPM), computing WACC, calculating ROIC, forecasting future FCFFs, and estimating the terminal value. These inputs are then used to arrive at the fair value per share, helping us assess whether a stock is undervalued or overvalued. 


In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import textwrap
import datetime as datetime

def format_currency(value, prefix='R'):
    try:
        return f"{prefix}{value:,.2f}"
    except:
        return f"{prefix}0.00"

def format_percentage(value):
    try:
        return f"{value*100:.2f}%"
    except:
        return "0.00%"
        
def calculate_dcf(ticker, growth_rate=0.00, perpetual_growth_rate=0.007, risk_free_rate=0.09, market_return=0.11, forecast_years=5):
    """
    Calculate the intrinsic value per share using a DCF model. 

    Parameters:
        ticker (str): Company ticker symbol 
        growth_rate (float): Annual growth rate for FCFF projections
        perpetual_growth_rate (float): Growth rate for terminal value
        risk_free_rate (float): Risk-free rate
        market_return (float): Expected market return
        forecast_years (int): Number of years for explicit forecast

    Returns:
        dict: Results including FCFF, WACC, ROIC, and fair value per share.

    """

    try:
        company = yf.Ticker(ticker)
        financials = company.financials.bfill(axis=1).ffill(axis=1)
        balance_sheet = company.balance_sheet.bfill(axis=1).ffill(axis=1)
        cashflow = company.cashflow.bfill(axis=1).ffill(axis=1)
        info = company.info

        name = info.get('shortName', 'N/A')
        sector = info.get('sector', 'N/A')
        industry = info.get('industry', 'N/A')
        country = info.get('country', 'N/A')
        website = info.get('website', 'N/A')
        description = info.get('longBusinessSummary', 'No description available.')
        desc_snippet = textwrap.shorten(description, width=300, placeholder="...")

        ebit = financials.loc['EBIT'].iloc[0] if 'EBIT' in financials.index else 0
        interest_expense = financials.loc['Interest Expense'].iloc[0] if 'Interest Expense' in financials.index else 0
        income_before_tax = financials.loc['Pretax Income'].iloc[0] if 'Pretax Income' in financials.index else 0
        taxes = financials.loc['Tax Provision'].iloc[0] if 'Tax Provision' in financials.index else 0

        total_debt = balance_sheet.loc['Total Debt'].iloc[0] if 'Total Debt' in balance_sheet.index else 0
        cash_equivalents = balance_sheet.loc['Cash And Cash Equivalents'].iloc[0] if 'Cash And Cash Equivalents' in balance_sheet.index else 0
        current_assets = balance_sheet.loc['Current Assets'].iloc[0] if 'Current Assets' in balance_sheet.index else 0
        current_liabilities = balance_sheet.loc['Current Liabilities'].iloc[0] if 'Current Liabilities' in balance_sheet.index else 0
        net_ppe = balance_sheet.loc['Net PPE'].iloc[0] if 'Net PPE' in balance_sheet.index else 0

        depreciation = cashflow.loc['Depreciation And Amortization'].iloc[0] if 'Depreciation And Amortization' in cashflow.index else 0
        capex = cashflow.loc['Capital Expenditure'].iloc[0] if 'Capital Expenditure' in cashflow.index else 0
        working_capital_change = cashflow.loc['Change In Working Capital'].iloc[0] if 'Change In Working Capital' in cashflow.index else 0

        beta = info.get('beta', 1.0)
        market_cap = info.get('marketCap', 0)
        shares_outstanding = info.get('sharesOutstanding', 1)
        current_price = info.get('currentPrice', 0) / 100

        fcff = ebit - taxes + depreciation - capex - working_capital_change

        if np.isnan(fcff) or fcff <= 0:
            net_income = financials.loc['Net Income'].iloc[0] if 'Net Income' in financials.index else 0
            effective_tax_rate = taxes / income_before_tax if income_before_tax != 0 else 0.25
            nopat = net_income + interest_expense * (1 - effective_tax_rate)
            fcff = nopat + depreciation - capex - working_capital_change

        effective_tax_rate = taxes / income_before_tax if income_before_tax != 0 else 0.25
        cost_of_debt = (interest_expense / total_debt) * (1 - effective_tax_rate) if total_debt != 0 else 0
        cost_of_equity = risk_free_rate + beta * (market_return - risk_free_rate)

        total_weight = total_debt + market_cap
        weight_debt = total_debt / total_weight if total_weight != 0 else 0
        weight_equity = market_cap / total_weight if total_weight != 0 else 1

        wacc = (weight_equity * cost_of_equity) + (weight_debt * cost_of_debt)
        wacc = min(max(wacc, 0.05), 0.25) 

        invested_capital = current_assets - current_liabilities + net_ppe
        roic = (ebit * (1 - effective_tax_rate)) / invested_capital if invested_capital != 0 else 0
        excess_returns = roic - wacc if roic != 0 and wacc != 0 else 0

        future_fcff = [fcff * (1 + growth_rate) ** t for t in range(1, forecast_years + 1)]
        last_fcff = future_fcff[-1] if future_fcff else fcff
        terminal_value = (last_fcff * (1 + perpetual_growth_rate)) / (wacc - perpetual_growth_rate) if wacc > perpetual_growth_rate else 0

        pv_fcff = [fcff / (1 + wacc) ** t for t, fcff in enumerate(future_fcff, 1)]
        pv_terminal = terminal_value / (1 + wacc) ** forecast_years if terminal_value != 0 else 0

        total_pv = sum(pv_fcff) + pv_terminal
        market_equity_value = total_pv + cash_equivalents - total_debt
        fair_value_per_share = market_equity_value / shares_outstanding if shares_outstanding != 0 else 0

        upside = ((fair_value_per_share - current_price) / current_price) * 100 if current_price else 0
        margin_of_safety_pct = ((fair_value_per_share - current_price) / fair_value_per_share) * 100 if fair_value_per_share != 0 else 0
        valuation = "Undervalued" if upside > 0 else "Overvalued"

        output = f"\n{'='*80}\n"
        output += f"DCF Analysis for {ticker} - {name}\n"
        output += f"Sector: {sector}\nIndustry: {industry}\nCountry: {country}\nWebsite: {website}\n"
        output += f"Description: {desc_snippet}\n\n"
        output += f"FCFF: {format_currency(fcff)}\n"
        output += f"WACC: {format_percentage(wacc)}\n"
        output += f"ROIC: {format_percentage(roic)}\n"
        output += f"Excess Returns: {format_percentage(excess_returns)}\n"
        output += f"Future FCFF ({2025+1}-{2025+forecast_years}): {[format_currency(x, prefix='T') for x in future_fcff]}\n"
        output += f"PV of FCFF: {[format_currency(x) for x in pv_fcff]}\n"
        output += f"Terminal Value: {format_currency(terminal_value)}\n"
        output += f"PV of Terminal Value: {format_currency(pv_terminal)}\n"
        output += f"Market Equity Value: {format_currency(market_equity_value)}\n"
        output += f"Fair Value Per Share: {format_currency(fair_value_per_share)}\n"
        output += f"Current Price: {format_currency(current_price)}\n"
        output += f"Upside: {upside:.2f}%\n"
        output += f"Margin of Safety: {margin_of_safety_pct:.2f}%\n"
        output += f"Valuation: {valuation}\n"
        output += f"{'='*80}\n"

        with open("jse_dcf_reports.txt", "a", encoding= "utf-8") as f:
            f.write(output + "\n")

        return output

    except Exception as e:
        return f"Error fetching data or calculating DCF for {ticker}: {e}"
       
tickers = ["ABG.JO", "ADH.JO", "AEL.JO", "AFE.JO", "AFH.JO", "AFT.JO", "AGL.JO", "AHR.JO", "AIP.JO", "ANG.JO", "ANH.JO", "APN.JO", "ARI.JO",
          "ARL.JO", "ATT.JO", "AVI.JO", "BAW.JO", "BHG.JO", "BID.JO", "BLU.JO", "BOX.JO", "BTI.JO", "BTN.JO", "BVT.JO", "BYI.JO", "CFR.JO", "CLS.JO",
          "CML.JO", "COH.JO", "CPI.JO", "CSB.JO", "DCP.JO", "DRD.JO", "DSY.JO", "DTC.JO", "EMI.JO", "EQU.JO", "EXX.JO", "FBR.JO", "FFB.JO", "FSR.JO",
          "FTB.JO", "GFI.JO", "GLN.JO", "GND.JO", "GRT.JO", "HAR.JO", "HCI.JO", "HDC.JO", "HMN.JO", "HYP.JO", "IMP.JO", "INL.JO", "INP.JO", "ITE.JO",
          "JSE.JO", "KAP.JO", "KIO.JO", "KRO.JO", "KST.JO", "LHC.JO", "LTE.JO", "MCG.JO", "MKR.JO", "MNP.JO", "MRP.JO", "MSP.JO", "MTH.JO", "MTM.JO",
          "MTN.JO", "N91.JO", "NED.JO", "NPH.JO", "NPN.JO", "NRP.JO", "NTC.JO", "NY1.JO", "OCE.JO", "OMN.JO", "OMU.JO", "OUT.JO", "PAN.JO", "PHP.JO",
          "PIK.JO", "PMR.JO", "PPC.JO", "PPH.JO", "PRX.JO", "QLT.JO", "RBX.JO", "RCL.JO", "RDF.JO", "REM.JO", "RES.JO", "RLO.JO", "RNI.JO", "S32.JO",
          "SAC.JO", "SAP.JO", "SBK.JO", "SHC.JO", "SHP.JO", "SLM.JO", "SNT.JO","SOL.JO", "SPG.JO", "SPP.JO", "SRE.JO", "SRI.JO", "SSS.JO","SSU.JO",
          "SSW.JO", "SUI.JO", "TBS.JO", "TFG.JO", "TGA.JO", "TKG.JO", "TRU.JO", "TSG.JO", "VAL.JO", "VKE.JO", "VOD.JO", "WBC.JO", "WHL.JO"]

for ticker in tickers:
    print(calculate_dcf(ticker))

C:\Users\kabel\AppData\Local\Temp\ipykernel_7656\2364596745.py:38: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  financials = company.financials.bfill(axis=1).ffill(axis=1)
C:\Users\kabel\AppData\Local\Temp\ipykernel_7656\2364596745.py:39: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  balance_sheet = company.balance_sheet.bfill(axis=1).ffill(axis=1)
C:\Users\kabel\AppData\Local\Temp\ipykernel_7656\2364596745.py:40: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infe


DCF Analysis for ABG.JO - Absa Group Limited
Sector: Financial Services
Industry: Banks - Regional
Country: South Africa
Website: https://www.absa.africa
Description: Absa Group Limited, together with its subsidiaries, provides retail and business banking, corporate and investment banking, insurance, financial, and wealth management products and services in South Africa and internationally. The company offers life and non-life insurance products; residential...

FCFF: R81,652,242,091.32
WACC: 22.98%
ROIC: 0.00%
Excess Returns: 0.00%
Future FCFF (2026-2030): ['T81,652,242,091.32', 'T81,652,242,091.32', 'T81,652,242,091.32', 'T81,652,242,091.32', 'T81,652,242,091.32']
PV of FCFF: ['R66,394,601,233.18', 'R53,988,022,373.98', 'R43,899,752,475.62', 'R35,696,589,404.05', 'R29,026,279,721.94']
Terminal Value: R369,043,582,332.18
PV of Terminal Value: R131,190,056,463.84
Market Equity Value: R252,273,301,672.61
Fair Value Per Share: R304.14
Current Price: R180.45
Upside: 68.55%
Margin of Safe

C:\Users\kabel\AppData\Local\Temp\ipykernel_7656\2364596745.py:38: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  financials = company.financials.bfill(axis=1).ffill(axis=1)
C:\Users\kabel\AppData\Local\Temp\ipykernel_7656\2364596745.py:39: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  balance_sheet = company.balance_sheet.bfill(axis=1).ffill(axis=1)
C:\Users\kabel\AppData\Local\Temp\ipykernel_7656\2364596745.py:40: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infe


DCF Analysis for ADH.JO - ADvTECH Ltd
Sector: Consumer Defensive
Industry: Education & Training Services
Country: South Africa
Website: https://www.advtech.co.za
Description: ADvTECH Limited provides education, training, and staff placement services in South Africa and other African countries. It operates through Schools, Tertiary, and Resourcing divisions. The Schools division offers pre-primary, primary, and secondary education services through face to face and...

FCFF: R2,814,500,000.00
WACC: 8.89%
ROIC: 23.26%
Excess Returns: 14.37%
Future FCFF (2026-2030): ['T2,814,500,000.00', 'T2,814,500,000.00', 'T2,814,500,000.00', 'T2,814,500,000.00', 'T2,814,500,000.00']
PV of FCFF: ['R2,584,718,539.15', 'R2,373,696,900.56', 'R2,179,903,494.48', 'R2,001,931,773.22', 'R1,838,490,022.51']
Terminal Value: R34,605,637,745.70
PV of Terminal Value: R22,605,123,367.63
Market Equity Value: R30,939,564,097.55
Fair Value Per Share: R56.45
Current Price: R32.62
Upside: 73.05%
Margin of Safety: 42.21%

C:\Users\kabel\AppData\Local\Temp\ipykernel_7656\2364596745.py:38: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  financials = company.financials.bfill(axis=1).ffill(axis=1)
C:\Users\kabel\AppData\Local\Temp\ipykernel_7656\2364596745.py:39: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  balance_sheet = company.balance_sheet.bfill(axis=1).ffill(axis=1)
C:\Users\kabel\AppData\Local\Temp\ipykernel_7656\2364596745.py:40: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infe

...


DCF Analysis for VOD.JO - Vodacom Group Ltd
Sector: Communication Services
Industry: Telecom Services
Country: South Africa
Website: https://www.vodacom.com
Description: Vodacom Group Limited provides communications products and services. It offers voice, messaging, converged services, broadband, data connectivity, mobile financial services, and other value-added services. The company also provides mobile and fixed line connectivity solutions, as well as internet...

FCFF: R71,193,000,000.00
WACC: 9.07%
ROIC: 31.54%
Excess Returns: 22.48%
Future FCFF (2026-2030): ['T71,193,000,000.00', 'T71,193,000,000.00', 'T71,193,000,000.00', 'T71,193,000,000.00', 'T71,193,000,000.00']
PV of FCFF: ['R65,274,277,885.17', 'R59,847,616,386.87', 'R54,872,107,409.46', 'R50,310,243,804.73', 'R46,127,636,629.74']
Terminal Value: R856,787,013,113.74
PV of Terminal Value: R555,132,667,677.82
Market Equity Value: R779,515,549,793.79
Fair Value Per Share: R402.76
Current Price: R139.08
Upside: 189.59%
Margin 

C:\Users\kabel\AppData\Local\Temp\ipykernel_7656\2364596745.py:38: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  financials = company.financials.bfill(axis=1).ffill(axis=1)
C:\Users\kabel\AppData\Local\Temp\ipykernel_7656\2364596745.py:39: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  balance_sheet = company.balance_sheet.bfill(axis=1).ffill(axis=1)
C:\Users\kabel\AppData\Local\Temp\ipykernel_7656\2364596745.py:40: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infe


DCF Analysis for WBC.JO - We Buy Cars Hlds Ltd
Sector: Consumer Cyclical
Industry: Auto & Truck Dealerships
Country: South Africa
Website: https://www.webuycars.co.za
Description: We Buy Cars Holdings Limited engages in buying, distributing, and retailing pre-owned motor vehicles through their website in South Africa. The company is also involved in the sale of car finance, insurance, and other ancillary products. In addition, it engages in the operation of car...

FCFF: R1,137,138,000.00
WACC: 10.72%
ROIC: 13.56%
Excess Returns: 2.84%
Future FCFF (2026-2030): ['T1,137,138,000.00', 'T1,137,138,000.00', 'T1,137,138,000.00', 'T1,137,138,000.00', 'T1,137,138,000.00']
PV of FCFF: ['R1,027,028,738.12', 'R927,581,374.41', 'R837,763,515.48', 'R756,642,734.79', 'R683,376,892.80']
Terminal Value: R11,426,815,248.19
PV of Terminal Value: R6,867,083,413.68
Market Equity Value: R9,843,506,669.29
Fair Value Per Share: R23.57
Current Price: R57.58
Upside: -59.07%
Margin of Safety: -144.32%
Valuatio

C:\Users\kabel\AppData\Local\Temp\ipykernel_7656\2364596745.py:38: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  financials = company.financials.bfill(axis=1).ffill(axis=1)
C:\Users\kabel\AppData\Local\Temp\ipykernel_7656\2364596745.py:39: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  balance_sheet = company.balance_sheet.bfill(axis=1).ffill(axis=1)
C:\Users\kabel\AppData\Local\Temp\ipykernel_7656\2364596745.py:40: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infe


DCF Analysis for WHL.JO - Woolworths Holdings Ltd
Sector: Consumer Cyclical
Industry: Department Stores
Country: South Africa
Website: https://www.woolworthsholdings.co.za
Description: Woolworths Holdings Limited, through its subsidiaries, operates a chain of retail stores in South Africa, Australia, and New Zealand. It operates through seven segments: Woolworths Fashion, Beauty and Home; Woolworths Food; Woolworths Financial Services; Woolworths Logistics; Country Road Group;...

FCFF: R11,849,000,000.00
WACC: 8.72%
ROIC: 27.11%
Excess Returns: 18.38%
Future FCFF (2026-2030): ['T11,849,000,000.00', 'T11,849,000,000.00', 'T11,849,000,000.00', 'T11,849,000,000.00', 'T11,849,000,000.00']
PV of FCFF: ['R10,898,176,894.10', 'R10,023,652,596.43', 'R9,219,304,508.47', 'R8,479,501,339.68', 'R7,799,063,682.47']
Terminal Value: R148,691,929,417.28
PV of Terminal Value: R97,869,679,010.45
Market Equity Value: R128,482,378,031.60
Fair Value Per Share: R144.09
Current Price: R50.00
Upside: 188.18

**Explanation**

This value and momentum investing is an innovative investment strategy that merges the foundational principles of value and investing with the dynamics aspects of momentum investing. At its core, value investing seeks to identify stocks that are undervalued relative to their intrinsic worth. Momentum investing, on the other hand, leverages the tendency of stocks to continue moving in the same direction-upward or downward based on recent price movements. This hybrid approach aims to invest in undervalued stocks that are currently experiencing increasing demand, thereby maximizing potential returns while minimizing risk. By harnessing the strengths of both strategies, Amare Capital Management (Pty) Ltd can capitalize on market inefficiencies and identify opportunities that others may overlook.

### FACTOR ALIGNED REVERSAL MODEL WITH VOLATILITY AND RISK MANAGEMENT FILTERS

This strategy identifies potential bullish reversals using the hammer candlestick pattern, filtered by the asset's position relative to its above 200-day moving average to confirm the trend and volatility conditions measured by the True Range Delta. It aims to enter long positions when a hammer candle forms under specific conditions and manages risk with stop-losses, profit-targets, and special situation handling. The approach is backtested across multiple tickers to ensure robustness, reflecting Amare Capital Management's commitment to rigorous statistical validation. 

In [ ]:
from typing import Callable, List 
import pandas as pd
import os
import yfinance as yf
from derivative_columns.atr import add_tr_delta_col_to_ohlc
from utils.import_data import get_local_ticker_data_file_name 

MUST_HAVE_DERIVATIVE_COLUMNS = {"tr", "tr_delta"}

def import_yahoo_finance_daily(ticker:str) -> pd.DataFrame:
    stock = yf.Ticker(ticker)
    end_date = datetime.today().strftime('%Y-%m-%d')
    df = stock.history(start="2020-01-01", end= end_date, interval="1d")
    df = df[["Open", "High", "Low", "Close", "Volume"]]
    df.index = pd.to.datetime(df.index).tz_localize(None)
    return df 

class TickersData:
    def __init__(self, tickers: list[str], add_features_cols_func: Callable, import_ohlc_func: Callable = import_yahoo_finance_daily):
        self.tickers_data_with_features = {}
        self.add_features_cols_func = add_features_cols_func
        self.import_ohlc_func = add_features_cols_func
        for ticker in tickers:
            df = self.get_df_with_features(ticker=ticker)
            for col in MUST_HAVE_DERIVATIVE_COLUMNS:
                if col not in df.columns:
                    df = add_tr_delta_col_to_ohlc(ohlc_df=df)
            self.tickers_data_with_features[ticker] = df
            
    def get_df_with_features(self, ticker: str) -> pd.DataFrame:
        filename_with_features = get_local_ticker_data_file_name(ticker, "with_features")
        filename_raw = get_local_ticker_data_file_name(ticker, "raw")
        if os.path.exists(filename_with_features):
            return pd.read_excel

**Explanation**

The implementation of the TickersData class enables efficient data retrieval from Yahoo Finance, with local caching in Excel files to ensure data integrity and reduce redundancy. By integrating key technical indicators such as the 200-day Moving Average (MA200), Average True Range (ATR), and hammer candle pattern detection, the firm streamlines its data management process, allowing greater focus on the development and refinement of investing strategies. 

In [ ]:
import pandas as pd
from constants2 import FEATURE_COL_NAME_ADVANCED, FEATURE_COL_NAME_BASIC 
from derivative_columns.atr import add_atr_col_to_df 
from derivative_columns.ma import add_moving_average 
from derivative_columns.hammer import add_col_is_hammer
from derivative_columns.shooting_star import add_col_is_shooting_star

MOVING_AVERAGE_N = 200
REQUIRED_DERIVATIVE_COLUMNS_F_V1_BASIC = {"atr_14", f"ma_{MOVING_AVERAGE_N}", "is_hammer", "is_shooting_star"}

def add_required_cols_for_f_v1_basic(df: pd.DataFrame) -> pd.DataFrame:
    df_columns = df.columns 
    internal_df = df.copy()
    if f"ma_{MOVING_AVERAGE_N}" not in df_columns:
        internal_df = add_moving_average(df=internal_df, n=MOVING_AVERAGE_N)
    if "atr_14" not in df_columns:
        internal_df = add_atr_col_to_df(df=internal_df, n=14, exponential=False)
    if "is_hammer" not in df_columns:
        internal_df = add_col_is_hammer(df=internal_df)
    if "is_shooting_star" not in df_columns:
        internal_df = add_col_is_shooting_star(df=internal_df)
    return internal_df 

def add_features_v1_basic(df: pd.DataFrame, atr_multiplier_threshold: int = 6) -> pd.DataFrame:
    res = df.copy()
    for col in REQUIRED_DERIVATIVE_COLUMNS_F_V1_BASIC:
        if col not in res.columns:
            res = add_required_cols_for_f_v1_basic(df=res)
    res[FEATURE_COL_NAME_BASIC] = res["Close"] < res[f"ma_{MOVING_AVERAGE_N}"]
    res[FEATURE_COL_NAME_ADVANCED] = (res["ma_200"] - res["Close"]) >= (res["atr_14"] * atr_multiplier_threshold)
    return res                                                    

**Explanation**

The add_features_v1_basic function is enhanced to incorporate a hammer candle signal (is_hammer) and to refine FEATURE_COL_NAME_ADVANCED to activate when the stock price is significantly below the 200-day Moving Average (MA200) with a confirmed hammer pattern. This transformation of complex market data into clear, actionable signals supports the firm's mission to elevate investing decisions through statistically grounded methodologies. 

In [ ]:
from typing import Optional, Tuple
from backtesting.backtesting import Strategy 
from backtesting import set_bokeh_output
set_bokeh_output(notebook=False)
from constants2 import DPS_STUB, FEATURE_COL_NAME_ADVANCED
from utils.strategy_exec.misc import get_current_position_size 

def get_desired_current_position_size(strategy: Strategy) -> Tuple[Optional[float], float, str]:
    current_position_size = (
        get_current_position_size(
            shares_count=strategy.position.size,
            equity=strategy.equity,
            last_price=strategy._data.Open[-1],
        )
        if strategy.position.size != 0 
        else 0
    )
    is_hammer = strategy._data["is_hammer"][-1]
    price_below_ma200 = strategy._data[FEATURE_COL_NAME_ADVANCED][-1]
    volatility_ok = strategy.data["tr_delta"][-1] < 2.5 

    desired_position_size: Optional[float] = None 
    message = DPS_STUB

    if current_position_size != 0:
        desired_position_size = current_position_size
        message = "Maintain existing position"
        return desired_position_size, current_position_size, message

    if is_hammer and price_below_ma200 and volatility_ok:
        desired_position_size = 1.0
        message = "Enter Long: Hammer reversal below MA200 with moderate volatility"

    return desired_position_size, current_position_size, message 

**Explanation**

The get_desired_current_position_size function is modified to initiate a 100% when a hammer candle forms, the price is at least six times the ATR_14 below the 200-day Moving Average (MA200), and volatility (True Range Delta) remains moderate (below 2.5). The function also represents existing entries or exits under specific conditions, reinforcing the firm's disciplined capital allocation framework - an essential pillar of its systematic and data-driven investment strategy.

In [ ]:
from backtesting import Strategy 
from constants2 import SL_TIGHTENED 
import numpy as np

def _get_n_atr(strategy: Strategy) -> float:
    index = len(strategy.data) - 1
    if strategy.data.tr_delta[index] > 1.98 and strategy.trades and strategy.trades[-1].pl > 0:
        return 1.1 
    return strategy_parameters.stop_loss_default_atr_multiplier

def update_stop_loss(strategy: Strategy):
    if not strategy.trades:
        return 
    n_atr = _get_n_atr(strategy)
    index = len(strategy.data) - 1
    for trade in strategy.trades:
        if trade.is_long:
            sl_price = max(trade.sl or -np.inf, strategy.data.Open[index] - strategy.data.atr_14[index] * n_atr)
        else:
            sl_price = min(trade.sl or np.inf, strategy.data.Open[index] + strategy.data.atr_14[index] * n_atr)
        if sl_price < 0:
            sl_price = None 
        if sl_price and trade.sl != sl-price: 
            trade.sl = sl-price 
            if n_atr == 1.1 and SL_TIGHTENED not in (trade.tag or ""):
                setattr(trade, f"_{trade.__class__.__qualname__}__tag", (trade.tag or "") + SL_TIGHTENED)

def check_set_profit_targets_long_trades(strategy: Strategy):
    last_price = strategy._data.Open[-1]
    min_profit_target_long = None 
    trades_long = [trade for trade in strategy.trades if trade.is_long]
    for trade in trades_long:
        if trade.tp is not None:
            min_profit_target_long = min(min_profit_target_long or trade.tp, trade.tp)
        if trades_long and min_profit_target_long  is None:
            min_profit_target_long = (float(strategy.parameters.profit_target_long_pct + 100) / 100) * last_price
            for trade in trades_long:
                if trade.tp is None:
                    trade.tp = min_profit_target_long

**Explanation**

The existing function such as update_stop_losses and check_set_profit_targets_long_trades are utilized with default parameters - a 2.5 ATR multiplier for stop-losses and a 29.9% profit target - to manage trades effectively. During periods of elevated volatility (when true range delta exceeds 1.98), stop-losses are tightened to 1.1 ATR, reinforcing the firm's commitment to robust risk management practice that underpin its pursuit of consistent and sustainable performance. 

In [ ]:
from backtesting import Strategy 
from constants2 import CLOSED_VOLATILITY_SPIKE, CLOSED_MAX_DURATION, SS_VOLATILITY_SPIKE, SS_MAX_DURATION, SS_NO_TODAY
from utils.strategy_exec.misc import add_tag_to_trades_and_close_position

def process_volatility_spike(strategy: Strategy) -> bool:
    if strategy.data.tr_delta[-1] < 2.5:
        return False 
    add_tag_to_trades_and_close_position(strategy, CLOSED_VOLATILITY_SPIKE)
    return True 

def process_max_duration(strategy: Strategy) -> bool:
    max_trade_duration_long = strategy.parameters.max_trade_duration_long
    if max_trade_duration_long is None or not strategy.trades:
        return False
    max_trade_duration = max((strategy.data.index[-1] - trade.entry_time).days for trade in strategy.trades)
    if strategy.trades[-1].is_long and max_trade_duration > max_trade_duration_long:
        add_tag_to_trades_and_close_position(strategy, CLOSED_MAX_DURATION)
        return True
    return False 

def process_special_situations(strategy: Strategy) -> Tuple[bool, str]: 
    if process_max_duration(strategy):
        return True, SS_MAX_DURATION 
    if process_volatility_spike(strategy):
        return True, SS_VOLATILITY_SPIKE
    return False, SS_NO_TODAY

**Explanation**

The process_special_situation function is employed to automatically close positions during extreme volatility spikes  (true range delta > 2.5) or when positions exceed a maximum holding period of 100 days. This proactive approach strengthens the resilience of the investing strategy, aligning with the firm's long-term vision of maintaining robust performance across varying market conditions. 

In [ ]:
from typing import List 
import pandas as pd
import numpy as np
from customizable.strategy_params import StrategyParams
from utils.local_data import TickersData
from strategy.run_backtest_for_ticker import run_backtest_for_ticker

def run_all_tickers(tickers_data: TickersData, strategy_params: StrategyParams, tickers: list[str]) -> float:
    open("app_run.log", "w", encoding="UTF-8").close()
    performance_res = pd.Dataframe()
    all_trades = pd.DataFrame()
    for ticker in tickers:
        ticker_data = tickers_data.get_data(ticker)
        stat, trades_df, last_day_result = run_backtest_for_ticker(ticker, ticker_data, strategy_params)
        stat = stat.drop(["_strategy", "_equity_curve", "_trades"])
        stat["SQN_MODIFIED"] =stat["SQN"] / np.sqrt(stat["# Trades"])
        performance_res[ticker] = stat
        if strategy_params.save_all_trades_in_xlsx:
            trades_df["Ticker"] = ticker
            all_trades = pd.concat([all_trades, trades_df])
    if len(tickers) > 1:
        performance_res.to_excel("output.xlsx")
    if strategy_params.save_all_trades_in_xlsx:
        all_trades.to_excel("all_trades.xlsx", index=False)
    return performance_res.loc["SQN_modified", :].mean()

**Explanation**

Comprehensive backtests are conducted across all tickers in tickers_all using the run_all_tickers function, while key parameters _ such as atr_multiplier_threshold are fine tuned through run_strategy_main_optimize. This rigorous testing process validates the strategy's effectiveness and reflects the firm's commitment to a data-driven, evidence-based investment approach.

In [ ]:
import logging 
from dotenv import load_dotenv 
from constants2 import LOG_FILE, tickers_all
from customizable.strategy_params import StrategyParams 
from f_v1_basic import add_features_v1_basic 
from strategy.all_tickers import run_all_tickers 
from utils.local_data import TickersData
import warnings 

logging.basicConfig(level=logging.DEBUG, format="%(message)s", filename=LOG_FILE, encoding="utf-8", filemode="a")

if __name__ == "__main__":
    load_dotenv()
    open(LOG_FILE, "w", encoding="UTF-8").close()

    strategy_params = StrategyParams(
        max_trade_duration_long=100,
        max_trade_duration_short=100,
        profit_target_long_pct=29.9,
        profit_target_short_pct=29.9,
        stop_loss_default_atr_multiplier=2.5,
        save_all_trades_in_xlsx= True,
    )

    tickers_data = TickersData (
        add_feature_cols_func=add_features_v1_basic,
        tickers=tickers_all,
    )

    SQN_modified_mean = run_all_tickers(
        tickers_data=tickers_data,
        tickers=tickers_all,
        strategy_params=strategy_params,
    )
    logging.debug(f"SQN_modified_mean={SQN_modified_mean}")
    print(f"SQN_modified_mean={SQN_modified_mean}, see output.xlsx") 
    
warnings.filterwarnings("ignore")

**Explanation**

The last_day_result function is used to monitor investing signals for real-time execution, with outcomes saved in output.xlsx for ongoing analysis. This continuos monitoring framework supports the firm's commitment to iterative strategy refinement and long-term performance improvement.

The System Quality Number (SQN) is a popular indicator of the investing system's quality. Its classic formula has a drawback: it tends to produce overly optimistic results when analyzing more than 100 orders, particularly when the number of orders exceeds 150-200.

SQN_modified is devoid of this drawback. It is simply the average of profits divided by the standard deviation of profits. An investing system is considered not bad if its SQN_modified has a positive value of at least 0.1. Systems whose value exceeds 0.2 are deemed decent or even good. 

In [ ]:
import pandas as pd
import yfinance as yf
from typing import List
from datetime import datetime
import logging
from dotenv import load_dotenv
from contextlib import redirect_stdout
import os

from f_v1_basic import add_features_v1_basic
from derivative_columns.shooting_star import add_col_is_shooting_star
from derivative_columns.atr import add_tr_delta_col_to_ohlc

LOG_FILE = "app_run.log"
logging.basicConfig(
    level=logging.DEBUG,
    format="%(message)s",
    filename=LOG_FILE,
    encoding="utf-8",
    filemode="a",
)

def fetch_ohlc_yfinance(ticker: str, start_date: str = "2020-01-01", end_date: str = None) -> pd.DataFrame:
    if end_date is None:
        end_date = end_date = datetime.today().strftime('%Y-%m-%d')

    try:
        df = yf.Ticker(ticker).history(start=start_date, end=end_date, interval="1d")
        if df.empty:
            logging.error(f"No data fetched for {ticker}")
            return pd.DataFrame()
        
        df = df[["Open", "High", "Low", "Close", "Volume"]]
        df.index = pd.to_datetime(df.index).tz_localize(None)  # Remove timezone
        logging.debug(f"Fetched {len(df)} rows for {ticker}")
        return df
    except Exception as e:
        logging.error(f"Error fetching data for {ticker}: {str(e)}")
        return pd.DataFrame()

def generate_bearish_signals(tickers: List[str], start_date: str = "2020-01-01", end_date: str = None) -> pd.DataFrame:
    if end_date is None:
        end_date = datetime.today().strftime('%Y-%m-%d')
   
    results = []
    
    with open(os.devnull, 'w') as devnull:
        with redirect_stdout(devnull):
            for ticker in tickers:
                logging.debug(f"Processing ticker: {ticker}")
           
                df = fetch_ohlc_yfinance(ticker, start_date, end_date)
                if df.empty:
                    logging.warning(f"Skipping {ticker} due to empty data")
                    continue
                
                df = add_features_v1_basic(df)
                
                df = add_col_is_shooting_star(df)
                df = add_tr_delta_col_to_ohlc(df)
                
                logging.debug(f"Data shape for {ticker}: {df.shape}")
                logging.debug(f"Columns for {ticker}: {list(df.columns)}")
                
                nan_counts = df[['Close', 'ma_200', 'atr_14', 'tr_delta', 'is_shooting_star']].isna().sum()
                logging.debug(f"NaN counts for {ticker}:\n{nan_counts}")
                
                df["Bearish_Signal"] = (
                    (df["is_shooting_star"] == True) &  
                    (df["Close"] > df["ma_200"]) &    
                    (df["tr_delta"] < 3.0)             
                )
                
                shooting_star_count = df["is_shooting_star"].sum()
                uptrend_count = (df["Close"] > df["ma_200"]).sum()
                volatility_count = (df["tr_delta"] < 3.0).sum()
                signal_count = df["Bearish_Signal"].sum()
                logging.debug(f"Shooting star count for {ticker}: {shooting_star_count}")
                logging.debug(f"Uptrend count (Close > ma_200) for {ticker}: {uptrend_count}")
                logging.debug(f"Volatility count (tr_delta < 3.0) for {ticker}: {volatility_count}")
                logging.debug(f"Bearish signal count for {ticker}: {signal_count}")
                
                df.to_excel(f"debug_{ticker}_full_data.xlsx")
                logging.debug(f"Saved full data for {ticker} to debug_{ticker}_full_data.xlsx")
                
                df_output = df[["Close", "ma_200", "atr_14", "tr_delta", "is_shooting_star", "Bearish_Signal"]].copy()
                df_output["Ticker"] = ticker
                df_output["Date"] = df_output.index
                df_output["Distance_to_MA200"] = ((df["Close"] - df["ma_200"]) / df["atr_14"]).round(2)
                df_output = df_output[["Ticker", "Date", "Close", "ma_200", "atr_14", "tr_delta", "Distance_to_MA200", "is_shooting_star", "Bearish_Signal"]]
                
                results.append(df_output[df_output["Bearish_Signal"] == True])
    
    result_df = pd.concat(results) if results else pd.DataFrame(
        columns=["Ticker", "Date", "Close", "ma_200", "atr_14", "tr_delta", "Distance_to_MA200", "is_shooting_star", "Bearish_Signal"]
    )
    
    result_df[["Close", "ma_200", "atr_14", "tr_delta"]] = result_df[["Close", "ma_200", "atr_14", "tr_delta"]].round(2)

    output_file = "bearish_signals.xlsx"
    result_df.to_excel(output_file, index=False)
    logging.debug(f"Bearish signals saved to {output_file}")
    
    return result_df

if __name__ == "__main__":
    load_dotenv()
    
    open(LOG_FILE, "w", encoding="utf-8").close()
    
    custom_tickers = ["ABG.JO", "ADH.JO", "AEL.JO", "AFE.JO", "AFH.JO", "AFT.JO", "AGL.JO", "AHR.JO", "AIP.JO", "ANG.JO", "ANH.JO", "APN.JO", "ARI.JO",
          "ARL.JO", "ATT.JO", "AVI.JO", "BAW.JO", "BHG.JO", "BID.JO", "BLU.JO", "BOX.JO", "BTI.JO", "BTN.JO", "BVT.JO", "BYI.JO", "CFR.JO", "CLS.JO",
          "CML.JO", "COH.JO", "CPI.JO", "CSB.JO", "DCP.JO", "DRD.JO", "DSY.JO", "DTC.JO", "EMI.JO", "EQU.JO", "EXX.JO", "FBR.JO", "FFB.JO", "FSR.JO",
          "FTB.JO", "GFI.JO", "GLN.JO", "GND.JO", "GRT.JO", "HAR.JO", "HCI.JO", "HDC.JO", "HMN.JO", "HYP.JO", "IMP.JO", "INL.JO", "INP.JO", "ITE.JO",
          "JSE.JO", "KAP.JO", "KIO.JO", "KRO.JO", "KST.JO", "LHC.JO", "LTE.JO", "MCG.JO", "MKR.JO", "MNP.JO", "MRP.JO", "MSP.JO", "MTH.JO", "MTM.JO",
          "MTN.JO", "N91.JO", "NED.JO", "NPH.JO", "NPN.JO", "NRP.JO", "NTC.JO", "NY1.JO", "OCE.JO", "OMN.JO", "OMU.JO", "OUT.JO", "PAN.JO", "PHP.JO",
          "PIK.JO", "PMR.JO", "PPC.JO", "PPH.JO", "PRX.JO", "QLT.JO", "RBX.JO", "RCL.JO", "RDF.JO", "REM.JO", "RES.JO", "RLO.JO", "RNI.JO", "S32.JO",
          "SAC.JO", "SAP.JO", "SBK.JO", "SHC.JO", "SHP.JO", "SLM.JO", "SNT.JO", "SOL.JO", "SPG.JO", "SPP.JO", "SRE.JO", "SRI.JO", "SSS.JO",
          "SSU.JO", "SSW.JO", "SUI.JO", "TBS.JO", "TFG.JO", "TGA.JO", "TKG.JO", "TRU.JO", "TSG.JO", "VAL.JO", "VKE.JO", "VOD.JO", "WBC.JO", "WHL.JO"]
    
    bearish_signals_df = generate_bearish_signals(tickers=custom_tickers)

    pd.set_option('display.width', 1000)
    pd.set_option('display.max_columns', None)

    print(f"Generated bearish signals for {len(custom_tickers)} tickers.")
    print(f"Total signals: {len(bearish_signals_df)}")
    print(f"Results saved to bearish_signals.xlsx")
    if not bearish_signals_df.empty:
        print("\nSample of bearish signals:")
        print(bearish_signals_df.head())

**Explanation** 

The objective of this step is to generate bearish signals by detecting potential reversal points using the shooting star candlestick pattern in an uptrend, filtered by momentum and volatility conditions. The shooting star pattern is identified when is_shooting_star == True, with an uptrend filter where the price is above the 200-day moving average (Close > ma_200). The volatility filter ensures that the True Range delta (tr_delta) is below 2.5 for moderate volatility. Additionally, an optional condition checks if the price is significantly above the 200-day MA ((Close - ma_200) >= (atr_14 * 6)). The output is saved in a DataFrame (bearish_signals.xlsx) with columns for Ticker, Date, Close, MA200, ATR_14, TR_Delta, Is_Shooting_Star, and Bearish_Signal, helping investors identify stocks to avoid buying at potential peaks or consider for short positions. This approach focuses on real-time monitoring without backtesting.

### ANCHORED VOLUME WEIGHTED AVERAGE PRICE (AVWAPs)


This systematic investing strategy utilizes Anchored VWAPs to identify momentum in trends, support and resistance levels, and optimal entry and exit points. Primarily designed for daily (1d) low-frequency investing, it can also be adapted for intraday timeframes like 15-minute or 5-minute charts for mid-frequency investing. The strategy focuses on stocks, incorporating key indicators such as Anchored VWAPs, the Average True Range (ATR), significant price levels, and a 5-day Simple Moving Average (SMA). Anchored VWAP, a powerful tool in technical analysis, calculates the volume-weighted average price of an asset from a specific anchor point, such as key highs, lows, or market events, offering dynamic support and resistance levels. It is computed as the cumulative sum of the Typical Price (Open + High + Low + Close)/4 multiplied by volume, divided by total volume. This momentum approach helps confirm price trends, identify support and resistance zones, and generate investing signals based on price interactions with VWAP levels. The provided code framework efficiently fetches OHLC data, computes Anchored VWAPs, detects significant price levels, and visualizes them, ensuring a systematic and repeatable investing process.

In [ ]:
import pandas as pd
from import_ohlc.yahoo_finance import get_ohlc_from_yf
from misc.atr import add_atr_col_to_df
from misc.fill_min_max import fill_is_min_max
from constants import ATR_SMOOTHING_N

def prepare_data(ticker: str, period: str= "2y", interval: str = "1d") -> pd.DataFrame:
    try:
        df = get_ohlc_from_yf(ticker=ticker, period=period, interval=interval)
    except Exception as e:
        print(f"Failed to fetch data for {ticker} from Yahoo Finance: {e}")
        return pd.DataFrame()
        
    df = add_atr_col_to_df(df, n=ATR_SMOOTHING_N, exponential=False)

    df = fill_is_min_max(df)

    print(f"Prepared data for {ticker}:")
    print(df[["Open", "High", "Low", "Close", "Volume", f"atr_{ATR_SMOOTHING_N}", "is_min", "is_max"]].tail())
    return df
    
ticker_data = {}
tickers = ["ABG.JO", "ADH.JO", "AEL.JO", "AFE.JO", "AFH.JO", "AFT.JO", "AGL.JO", "AHR.JO", "AIP.JO", "ANG.JO", "ANH.JO", "APN.JO", "ARI.JO",
          "ARL.JO", "ATT.JO", "AVI.JO", "BAW.JO", "BHG.JO", "BID.JO", "BLU.JO", "BOX.JO", "BTI.JO", "BTN.JO", "BVT.JO", "BYI.JO", "CFR.JO", "CLS.JO",
          "CML.JO", "COH.JO", "CPI.JO", "CSB.JO", "DCP.JO", "DRD.JO", "DSY.JO", "DTC.JO", "EMI.JO", "EQU.JO", "EXX.JO", "FBR.JO", "FFB.JO", "FSR.JO",
          "FTB.JO", "GFI.JO", "GLN.JO", "GND.JO", "GRT.JO", "HAR.JO", "HCI.JO", "HDC.JO", "HMN.JO", "HYP.JO", "IMP.JO", "INL.JO", "INP.JO", "ITE.JO",
          "JSE.JO", "KAP.JO", "KIO.JO", "KRO.JO", "KST.JO", "LHC.JO", "LTE.JO", "MCG.JO", "MKR.JO", "MNP.JO", "MRP.JO", "MSP.JO", "MTH.JO", "MTM.JO",
          "MTN.JO", "N91.JO", "NED.JO", "NPH.JO", "NPN.JO", "NRP.JO", "NTC.JO", "NY1.JO", "OCE.JO", "OMN.JO", "OMU.JO", "OUT.JO", "PAN.JO", "PHP.JO",
          "PIK.JO", "PMR.JO", "PPC.JO", "PPH.JO", "PRX.JO", "QLT.JO", "RBX.JO", "RCL.JO", "RDF.JO", "REM.JO", "RES.JO", "RLO.JO", "RNI.JO", "S32.JO",
          "SAC.JO", "SAP.JO", "SBK.JO", "SHC.JO", "SHP.JO", "SLM.JO", "SNT.JO", "SOL.JO", "SPG.JO", "SPP.JO", "SRE.JO", "SRI.JO", "SSS.JO",
          "SSU.JO", "SSW.JO", "SUI.JO", "TBS.JO", "TFG.JO", "TGA.JO", "TKG.JO", "TRU.JO", "TSG.JO", "VAL.JO", "VKE.JO", "VOD.JO", "WBC.JO", "WHL.JO"]

for ticker in tickers: 
    df = prepare_data(ticker)
    if not df.empty:
        ticker_data[ticker] = df 

**Explanation**

The code fetches two years of daily OHLCV data for the ticker using the get_ohlc_from_yf function, computes a 14-period ATR (shifted to the previous day's value), and identifies significant highs and lows where price movements exceed 2.5 times the ATR. The resulting data, including OHLCV, ATR, and marked min/max levels, is stored in a dictionary of DataFrames for further analysis. Key functions from the codebase, such as add_atr_col_to_df for calculating volatility and fill_is_min_max for detecting significant levels, are used. This data provides the necessary foundation for VWAP-based investing and risk management. The output is a dictionary of DataFrames, printed for verification, containing the OHLCV, ATR, and key level indicators.



In [ ]:
from constants import first_day_of_year

def get_anchor_dates(df: pd.DataFrame, custom_dates: list[str] = None) -> list[str]:
    last_min_date = df[df["is_min"]].index.max()
    last_max_date = df[df["is_max"]].index.max()

    anchor_dates = [first_day_of_year]
    
    if pd.notna(last_min_date):
        anchor_dates.append(last_min_date.strftime('%Y-%m-%d %H:%M:%S'))
    if pd.notna(last_max_date):
        anchor_dates.append(last_max_date.strftime('%Y-%m-%d %H:%M:%S'))

    anchor_dates = [date for date in anchor_dates if pd.notna (date)]
    print(f"Anchor dates for {df.attrs.get('ticker', 'unknown')}: {anchor_dates}")
    return anchor_dates

anchor_dates_dict = {
    ticker: get_anchor_dates(df.assign(attrs={"ticker": ticker}))
    for ticker, df in ticker_data.items()
}

**Explanation**

The code creates a list of anchor dates for a specific stock ticker, starting with the fixed baseline date, first_day_of_year. It then adds custom_dates (such as earning reports, price peaks, and market corrections), and incorporates the most recent min/max dates from the stock's price history defined in prepare data function. The anchored dates are stored in a dictionary, providing a set of key reference points for further analysis, like VWAP calculations or investing signals. Custom dates are chosen to reflect significant events, such as earning reports or major price movements, which help us investors make data-driven decisions. The code automates the process of adding these anchor points, ensuring they align with key market events, while offering flexibility for our strategies. 

In [ ]:
import pandas as pd
from vwaps_plot import vwaps_plot_build_save
from misc.chart_annotation import get_chart_annotation_1d 
from constants import ATR_SMOOTHING_N, first_day_of_year
import matplotlib.pyplot as plt

def analyze_ticker(df: pd.DataFrame, ticker: str, anchor_dates: list[str]):
    df.attrs["ticker"] = ticker
    df = df.copy()
    
    df.index = pd.to_datetime(df.index)
    if df.index.tz is not None:
        df.index = df.index.tz_convert(None)
    
    dates_only = df.index.normalize().date

    for i, anchor_date in enumerate(anchor_dates, 1):
        anchor_ts = pd.Timestamp(anchor_date).date()
        if anchor_ts in dates_only:
            anchor_idx = list(dates_only).index(anchor_ts)
        else:
            if anchor_ts < df.index[0].date():
                print(f"Info: Anchor date {anchor_date} is before the start of data for {ticker}. Using first available date.")
            else:
                #print(f"Warning: Anchor date {anchor_date} not found in {ticker} index. Using start of data.")
                anchor_idx = 0

        df_from_anchor = df.iloc[anchor_idx:]
        typical_price = (df_from_anchor["Open"] + df_from_anchor["High"] +
                         df_from_anchor["Low"] + df_from_anchor["Close"]) / 4
        cumulative_typical_volume = (typical_price * df_from_anchor["Volume"]).cumsum()
        cumulative_volume = df_from_anchor["Volume"].cumsum()
        vwap = cumulative_typical_volume / cumulative_volume.replace(0, pd.NA)
        vwap = vwap.ffill()
        df.loc[df_from_anchor.index, f"A_VWAP_{i}"] = vwap

    vwaps_plot_build_save(
        input_df=df,
        anchor_dates=anchor_dates,
        chart_title=f"{ticker} Daily with Anchored VWAPs",
        chart_annotation_func=get_chart_annotation_1d,
        add_last_min_max=False,
        file_name=f"daily_{ticker}_annotated.png",
        print_df=False  
    )
   
    last_close = df["Close"].iloc[-1]
    vwap_year = df["A_VWAP_1"].iloc[-1]
    vwap_min = df["A_VWAP_2"].iloc[-1]
    vwap_max = df["A_VWAP_3"].iloc[-1]
    atr = df[f"atr_{ATR_SMOOTHING_N}"].iloc[-1]

    trend = "Neutral"
    if last_close > vwap_year and last_close > vwap_min:
        trend = "Bullish"
    elif last_close < vwap_year and last_close < vwap_max:
        trend = "Bearish"

    signal = None
    if trend == "Bullish" and last_close > vwap_min and abs(last_close - vwap_min) < atr * 0.5:
        signal = "Long"
    elif trend == "Bearish" and last_close < vwap_max and abs(vwap_max - last_close) < atr * 0.5:
        signal = "Short"

    if signal == "Long":
        entry_price = last_close
        stop_loss = vwap_min - atr
        take_profit = vwap_max
        risk = entry_price - stop_loss
    elif signal == "Short":
        entry_price = last_close
        stop_loss = vwap_max + atr
        take_profit = vwap_min
        risk = stop_loss - entry_price
    else:
        entry_price = stop_loss = take_profit = risk = None

    account_size = 10000
    risk_percent = 0.01
    position_size = (account_size * risk_percent) / risk if risk else 0

    return {
        "trend": trend,
        "signal": signal,
        "last_close": last_close,
        "vwap_year": vwap_year,
        "vwap_min": vwap_min,
        "vwap_max": vwap_max,
        "atr": atr,
        "entry_price": entry_price,
        "stop_loss": stop_loss,
        "take_profit": take_profit,
        "position_size": position_size
    }

results = {}
for ticker in ticker_data:
    results[ticker] = analyze_ticker(ticker_data[ticker], ticker, anchor_dates_dict[ticker])

results_df = pd.DataFrame(results).T

results_df.to_excel("anchored_vwap.xlsx", index=True)

display(results_df)

**Explanation**

This script computes and plots Anchored VWAPs for each ticker, then applies a VWAP-based investing strategy. It calculates three anchored VWAP levels—year-start, recent minimum, and recent maximum—and uses them to determine market trend and generate investing signals. A bullish trend is identified when the current close is above both the year-start and last-min VWAPs, while a bearish trend occurs when the close is below both the year-start and last-max VWAPs. Entry signals are triggered when the price is near key VWAPs within 0.5× ATR, and signal exits are defined using ATR-based stop losses and VWAP-based take profits. It outputs a structured dictionary with investing details and saves annotated charts for each ticker, turning the VWAP strategy into a fully operational, risk-managed system.


In [ ]:
from datetime import datetime

print("\nTrade Summary:")
for ticker, res in results.items():
    print(f"\nTicker: {ticker} on {datetime.now().date()}")
    print(f"Trend: {res['trend']}")
    print(f"Signal: {res['signal']}")
    print(f"Last Close: {res['last_close']:.2f}")
    print(f"VWAPs - Year: {res['vwap_year']:.2f}, Min: {res['vwap_min']:.2f}, Max: {res['vwap_max']:.2f}")
    print(f"ATR: {res['atr']:.2f}")
    if res["signal"]:
        print(f"Entry: {res['entry_price']:.2f}, Stop Loss: {res['stop_loss']:.2f}, "
              f"Take Profit: {res['take_profit']:.2f}")
        print(f"Position Size: {res['position_size']:.2f} shares")
        print(f"Executing {res['signal']} trade for {ticker}")
    else:
        print("No trade signal generated.")

**Explanation**

This step runs the VWAP-based strategy and prints a clear, date-stamped investing summary for each. For every ticker, it displays the detected trend momentum, signal (if any), last closing price, VWAP levels (year, min, and max), and ATR value. If an investing signal is generated, it also shows detailed trade parameters including entry price, stop loss, take profit, and calculated position size. The output provides a concise and actionable snapshot of investing setups, supporting informed execution and ongoing strategy monitoring.


In [ ]:
from draw_avg import draw_5_days_avg
from price_volume import draw_profile_of_data
from vwaps_plot import vwaps_plot_build_save
from misc.chart_annotation import get_chart_annotation_1d
from ratio import draw_ratio   
import matplotlib.pyplot as plt
import warnings 

for ticker in ticker_data:
    draw_5_days_avg(ticker=ticker, interval="15m")  
    print(f"{ticker}: 5-day SMA image generated")

    draw_profile_of_data(ohlc_df=ticker_data[ticker], ticker=ticker)
    print(f"{ticker}: Price and Volume profile image generated")

GENERATE_INTRADAY_VWAP = False # We can toggle this to true if we want to re-enable it 
    
intraday_df = get_ohlc_from_yf(ticker=ticker, period="5d", interval="1m")
intraday_df = add_atr_col_to_df(intraday_df, n=ATR_SMOOTHING_N, exponential=False)

if GENERATE_INTRADAY_VWAP:
    vwaps_plot_build_save(
        input_df=intraday_df,
        anchor_dates=anchor_dates_dict[ticker],
        chart_title=f"{ticker} 1m with Anchored VWAPs",
        chart_annotation_func=get_chart_annotation_1d,
        add_last_min_max=False,
        file_name=f"intraday_{ticker}.png",
        hide_extended_hours=True,
        print_df=False
    )
    print(f"{ticker}: Intraday VWAP image generated") 
    plt.close()

**Explanation**

The reason for integrating intraday tools such as SMA, volume profiles, relative strength ratios, and quick charting features is to enrich the daily investing strategy with more granular, real-time insights into market conditions. By incorporating these elements, investors can monitor price movements and momentum trends more effectively within the day, allowing for better short-term decision-making. The use of intraday VWAP and other indicators helps identify key price levels and momentum trends during trading hours, enhancing the strategy’s accuracy. 

### EXPONENTIAL MOVING AVERAGE CROSSOVER (MOMENTUM)

The EWMAC (Exponentially Weighted Moving Average Crossover) strategy is an investing rule that captures trends in asset prices using only price data. It compares fast and slow EWMA of the price to detect trends: when the fast EWMA is above the slow EWMA, it signals an uptrend (go long), and when the fast EWMA is below the slow EWMA, it signals a downtrend (go short). The raw signal is adjusted for volatility and scaled to reflect forecast strength, then capped to limit extremes. This simple systematic approach is backed by both empirical performance and behavioral finance theory, making it a robust and explainable investing strategy.

In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
import time

tickers = ["ABG.JO", "ADH.JO", "AEL.JO", "AFE.JO", "AFH.JO", "AFT.JO", "AGL.JO", "AHR.JO", "AIP.JO", "ANG.JO", "ANH.JO", "APN.JO", "ARI.JO",
          "ARL.JO", "ATT.JO", "AVI.JO", "BAW.JO", "BHG.JO", "BID.JO", "BLU.JO", "BOX.JO", "BTI.JO", "BTN.JO", "BVT.JO", "BYI.JO", "CFR.JO", "CLS.JO",
          "CML.JO", "COH.JO", "CPI.JO", "CSB.JO", "DCP.JO", "DRD.JO", "DSY.JO", "DTC.JO", "EMI.JO", "EQU.JO", "EXX.JO", "FBR.JO", "FFB.JO", "FSR.JO",
          "FTB.JO", "GFI.JO", "GLN.JO", "GND.JO", "GRT.JO", "HAR.JO", "HCI.JO", "HDC.JO", "HMN.JO", "HYP.JO", "IMP.JO", "INL.JO", "INP.JO", "ITE.JO",
          "JSE.JO", "KAP.JO", "KIO.JO", "KRO.JO", "KST.JO", "LHC.JO", "LTE.JO", "MCG.JO", "MKR.JO", "MNP.JO", "MRP.JO", "MSP.JO", "MTH.JO", "MTM.JO",
          "MTN.JO", "N91.JO", "NED.JO", "NPH.JO", "NPN.JO", "NRP.JO", "NTC.JO", "NY1.JO", "OCE.JO", "OMN.JO", "OMU.JO", "OUT.JO", "PAN.JO", "PHP.JO",
          "PIK.JO", "PMR.JO", "PPC.JO", "PPH.JO", "PRX.JO", "QLT.JO", "RBX.JO", "RCL.JO", "RDF.JO", "REM.JO", "RES.JO", "RLO.JO", "RNI.JO", "S32.JO",
          "SAC.JO", "SAP.JO", "SBK.JO", "SHC.JO", "SHP.JO", "SLM.JO", "SNT.JO", "SOL.JO", "SPG.JO", "SPP.JO", "SRE.JO", "SRI.JO", "SSS.JO",
          "SSU.JO", "SSW.JO", "SUI.JO", "TBS.JO", "TFG.JO", "TGA.JO", "TKG.JO", "TRU.JO", "TSG.JO", "VAL.JO", "VKE.JO", "VOD.JO", "WBC.JO", "WHL.JO"]

start_date = "2024-01-01"
end_date = datetime.today().strftime('%Y-%m-%d')
Lfast = 16
Lslow = 4 * Lfast
vol_lookback = 25
capmin = -20
capmax = 20

def ewmac_forecast_scalar(Lfast, Lslow):
    return 10 / np.sqrt(Lfast)

def retry_download(ticker, start, end, max_retries=3):
    for attempt in range(max_retries):
        try:
            df = yf.download(ticker, start=start, end=end)
            if not df.empty:
                return df
        except Exception as e:
            print(f"[Retry {attempt+1}] Error downloadding {ticker}: {e}")
    print(f"Failed to retrieve data for {ticker} after {max_retries} attempts.")
    return pd.DataFrame()
    

f_scalar = ewmac_forecast_scalar(Lfast, Lslow)

data = yf.download(tickers, start=start_date, end=end_date)

for ticker in tickers:
    try:
        if ticker not in data["Close"] or data["Close"][ticker].dropna().empty:
            print(f"No data for {ticker} in batch. Retrying individually...")
            single_data = retry_download(ticker, start_date, end_date)
            price = single_data["Close"].dropna()
        else:
            price = data["Close"][ticker].dropna()

        if price.empty:
            print(f"No data for {ticker}. Skipping...")
            continue
            
        fast_ewma = price.ewm(span=Lfast).mean()
        slow_ewma = price.ewm(span=Lslow).mean()
        raw_ewmac = fast_ewma - slow_ewma
        returns = price.pct_change()
        vol = returns.ewm(span=vol_lookback).std()
        vol_adj_ewmac = raw_ewmac / vol
        forecast = vol_adj_ewmac * f_scalar
        cap_forecast = forecast.clip(lower=capmin, upper=capmax)

        fig, axs = plt.subplots(1, 2, figsize=(18, 6))
        axs[0].plot(price, label='Price', color='black')
        axs[0].plot(fast_ewma, label=f'Fast EWMA ({Lfast})', linestyle='--')
        axs[0].plot(slow_ewma, label=f'Slow EWMA ({Lslow})', linestyle='--')
        axs[0].set_title(f"EWMAC Crossover\n{ticker}")
        axs[0].set_xlabel("Date")
        axs[0].set_ylabel("Price")
        axs[0].legend()
        axs[0].grid(True)
        
        axs[1].plot(cap_forecast, label='Capped Forecast Signal', color='blue')
        axs[1].axhline(10, color='green', linestyle='--', label='Buy Threshold')
        axs[1].axhline(-10, color='red', linestyle='--', label='Sell Threshold')
        axs[1].set_title("Capped EWMAC Forecast Signal")
        axs[1].set_xlabel("Date")
        axs[1].set_ylabel("Forecast Value")
        axs[1].legend()
        axs[1].grid(True)

        plt.tight_layout()
        plt.savefig(f"{ticker}_ewmac_combined.png")
        plt.close()

        print(f" Saved: {ticker}_ewmac_combined.png")

        time.sleep(0.2)

    except Exception as e:
        print(f" Error with {ticker}: {e}")

**Explanation**

The Exponentially Weighted Moving Average Crossover (EWMAC) strategy is a robust and intuitive momentum investing rule that captures medium- to long-term momentum in asset prices. By comparing a fast-moving average to a slow-moving average, the strategy identifies directional trends: it generates buy signals when prices are trending upward (fast MA > slow MA) and sell signals during downtrends (fast MA < slow MA).

The result is a dynamic signal that is responsive to trends, adaptive to volatility, and simple to implement, making it an ideal component of a systematic investing strategy. Its strength lies not only in its performance but also in its behavioral justification, simplicity, and positive skewness — offering large potential gains during strong market trends while limiting losses in range-bound periods.